### Code Description

This Python script implements a trading strategy that uses stock movement data to compute correlations between stocks. The key steps involved are:

1. **Data Retrieval**: The `get_stock_data` function uses the `yfinance` library to download daily OHLC (Open, High, Low, Close) data for a list of Nifty500 stocks over a specified date range.

2. **Movement Definition**: The `define_movement` function classifies daily stock movements as 'Up', 'Down', or 'Neutral' based on the closing price relative to the opening price. A movement is classified as 'Up' if the closing price is at least 0.5% higher than the opening price, 'Down' if it's at least 0.5% lower, and 'Neutral' otherwise.

3. **Data Preparation**:
   - Combine the OHLC data of all stocks into a single DataFrame, renaming columns to include stock symbols.
   - Calculate the movement direction for each stock using the `define_movement` function.

4. **Correlation Calculation**:
   - Create a DataFrame of stock movements and shift it to align movements from the previous day with the next day.
   - Compute the correlation between movements of different stocks using cross-tabulations and correlations of contingency tables.

5. **Results**:
   - Flatten the resulting correlations DataFrame and sort it to display the highest correlations between stock pairs.

The final output is a DataFrame showing the correlation between the movements of different stocks, indicating how similar or related their movement patterns are.

In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from itertools import permutations
from datetime import timedelta
import statistics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
data = yf.download("SBIN.NS", period="1mo", interval='1d', progress=False)
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-16,882.299988,888.099976,874.299988,880.700012,880.700012,11893692
2024-07-18,880.650024,894.900024,877.500000,893.549988,893.549988,19435183
2024-07-19,895.000000,899.000000,883.299988,889.349976,889.349976,14562742
2024-07-22,881.450012,895.000000,868.000000,876.799988,876.799988,14614111
2024-07-23,883.000000,884.500000,849.099976,863.900024,863.900024,18279212
2024-07-24,864.200012,868.000000,847.200012,852.000000,852.000000,19624845
2024-07-25,847.500000,850.750000,841.200012,848.500000,848.500000,9522375
2024-07-26,844.650024,864.250000,844.000000,862.450012,862.450012,12081506
2024-07-29,866.049988,889.099976,863.200012,871.599976,871.599976,26582781


In [3]:
# All Scrip baskets. Should change annually
scrip_Sector = ['^NSEI','^NSEBANK','^CNXIT','^CNXAUTO','^CNXPHARMA']
scrip_Test = ['ACC.NS','ABBOTINDIA.NS','ADANIENT.NS','ADANIGREEN.NS','ADANIPORTS.NS']
scrip_N50 = ['ADANIPORTS.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJFINANCE.NS', 'BHARTIARTL.NS', 'BPCL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GAIL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'INDUSINDBK.NS', 'INFY.NS', 'IOC.NS', 'ITC.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SBIN.NS', 'SHREECEM.NS', 'SUNPHARMA.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS', 'UPL.NS', 'WIPRO.NS']
scrip_N100 = ['ACC.NS','ABBOTINDIA.NS','ADANIENT.NS','ADANIGREEN.NS','ADANIPORTS.NS','ALKEM.NS','AMBUJACEM.NS','APOLLOHOSP.NS','ASIANPAINT.NS','AUROPHARMA.NS','DMART.NS','AXISBANK.NS','BAJAJ-AUTO.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BAJAJHLDNG.NS','BANDHANBNK.NS','BERGEPAINT.NS','BPCL.NS','BHARTIARTL.NS','BIOCON.NS','BOSCHLTD.NS','BRITANNIA.NS','CIPLA.NS','COALINDIA.NS','COLPAL.NS','DLF.NS','DABUR.NS','DIVISLAB.NS','DRREDDY.NS','EICHERMOT.NS','GAIL.NS','GODREJCP.NS','GRASIM.NS','HCLTECH.NS','HDFCAMC.NS','HDFCBANK.NS','HDFCLIFE.NS','HAVELLS.NS','HEROMOTOCO.NS','HINDALCO.NS','HINDPETRO.NS','HINDUNILVR.NS','ICICIBANK.NS','ICICIGI.NS','ICICIPRULI.NS','ITC.NS','IOC.NS','IGL.NS','INDUSTOWER.NS','INDUSINDBK.NS','NAUKRI.NS','INFY.NS','INDIGO.NS','JSWSTEEL.NS','JUBLFOOD.NS','KOTAKBANK.NS','LT.NS','LUPIN.NS','MRF.NS','M&M.NS','MARICO.NS','MARUTI.NS','MUTHOOTFIN.NS','NMDC.NS','NTPC.NS','NESTLEIND.NS','ONGC.NS','PETRONET.NS','PIDILITIND.NS','PEL.NS','POWERGRID.NS','PGHH.NS','PNB.NS','RELIANCE.NS','SBICARD.NS','SBILIFE.NS','SHREECEM.NS','SIEMENS.NS','SBIN.NS','SUNPHARMA.NS','TCS.NS','TATACONSUM.NS','TATAMOTORS.NS','TATASTEEL.NS','TECHM.NS','TITAN.NS','TORNTPHARM.NS','UPL.NS','ULTRACEMCO.NS','UBL.NS','VEDL.NS','WIPRO.NS','YESBANK.NS']
scrip_N500 = ['3MINDIA.NS', 'ABB.NS', 'POWERINDIA.NS', 'ACC.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ADVENZYMES.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKZOINDIA.NS', 'ALEMBICLTD.NS', 'APLLTD.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANGELONE.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASHOKA.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AVANTIFEED.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJAJCON.NS', 'BAJAJELEC.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BALAMINES.NS', 'BALKRISIND.NS', 'BALMLAWRIE.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARATRAS.NS', 'BHARTIARTL.NS', 'BIOCON.NS', 'BIRLACORPN.NS', 'BSOFT.NS', 'BLISSGVS.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'CCL.NS', 'CESC.NS', 'CRISIL.NS', 'CSBBANK.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CENTURYTEX.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCBBANK.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DEEPAKNTR.NS', 'DELTACORP.NS', 'DHANI.NS', 'DHANUKA.NS', 'DBL.NS', 'DISHTV.NS', 'DCAL.NS', 'DIVISLAB.NS', 'DIXON.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EPL.NS', 'EDELWEISS.NS', 'EICHERMOT.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'EXIDEIND.NS', 'FDC.NS', 'FEDERALBNK.NS', 'FINEORG.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FORTIS.NS', 'FCONSUMER.NS', 'FRETAIL.NS', 'GAIL.NS', 'GEPIL.NS', 'GMMPFAUDLR.NS', 'GMRINFRA.NS', 'GALAXYSURF.NS', 'GRSE.NS', 'GARFIBRES.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GESHIP.NS', 'GREAVESCOT.NS', 'GRINDWELL.NS', 'GUJALKALI.NS', 'GAEL.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GNFC.NS', 'GPPL.NS', 'GSFC.NS', 'GSPL.NS', 'GULFOILLUB.NS', 'HEG.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HATSUN.NS', 'HAVELLS.NS', 'HEIDELBERG.NS', 'HEMIPROP.NS', 'HEROMOTOCO.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HONAUT.NS', 'HUDCO.NS', 'HUHTAMAKI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'ISEC.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IDFC.NS', 'IFBIND.NS', 'IIFL.NS', 'IOLCP.NS', 'IRB.NS', 'IRCON.NS', 'ITC.NS', 'ITI.NS', 'INDIACEM.NS', 'IBULHSGFIN.NS', 'IBREALEST.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'ICIL.NS', 'INDOCO.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'INFIBEAM.NS', 'NAUKRI.NS', 'INFY.NS', 'INGERRAND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JKLAKSHMI.NS', 'JKPAPER.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWSTEEL.NS', 'JTEKTINDIA.NS', 'JAMNAAUTO.NS', 'JINDALSAW.NS',   'JINDALSTEL.NS', 'JCHAC.NS', 'JUBLFOOD.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'KPRMILL.NS', 'KEI.NS', 'KNRCON.NS', 'KPITTECH.NS', 'KRBL.NS', 'KSB.NS', 'KAJARIACER.NS',   'KARURVYSYA.NS', 'KSCL.NS', 'KEC.NS', 'KOTAKBANK.NS',  'LTTS.NS', 'LICHSGFIN.NS', 'LAOPALA.NS', 'LAXMIMACH.NS',  'LT.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LINDEINDIA.NS', 'LUPIN.NS', 'LUXIND.NS', 'MASFIN.NS', 'MMTC.NS', 'MOIL.NS', 'MRF.NS', 'MGL.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS',  'MHRIL.NS', 'MAHLOG.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MARICO.NS', 'MARUTI.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS',  'MINDACORP.NS',  'MIDHANI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NESCO.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NOCIL.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NFL.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWORK18.NS', 'NILKAMAL.NS', 'NAM-INDIA.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OFSS.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS',  'PAGEIND.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS',  'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PEL.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POLYPLEX.NS', 'PFC.NS', 'POWERGRID.NS', 'PRESTIGE.NS', 'PRINCEPIPE.NS', 'PRSMJOHNSN.NS', 'PGHL.NS', 'PGHH.NS', 'PNB.NS', 'QUESS.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAIN.NS', 'RAJESHEXPO.NS', 'RALLIS.NS', 'RCF.NS', 'RATNAMANI.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'RELAXO.NS', 'RELIANCE.NS', 'RESPONIND.NS', 'ROSSARI.NS', 'ROUTE.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SIS.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SANOFI.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SEQUENT.NS', 'SHARDACROP.NS', 'SFL.NS', 'SHILPAMED.NS', 'SCI.NS', 'SHOPERSTOP.NS', 'SHREECEM.NS',   'SIEMENS.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SOLARA.NS', 'SONATSOFTW.NS', 'SPANDANA.NS',   'SBIN.NS', 'SAIL.NS', 'SWSOLAR.NS', 'STLTECH.NS', 'STAR.NS', 'SUDARSCHEM.NS', 'SUMICHEM.NS', 'SPARC.NS', 'SUNPHARMA.NS', 'SUNTV.NS',  'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNTECK.NS', 'SUPRAJIT.NS', 'SUPREMEIND.NS',  'SUVENPHAR.NS', 'SUZLON.NS', 'SWANENERGY.NS', 'SYMPHONY.NS', 'SYNGENE.NS', 'TCIEXP.NS', 'TCNSBRANDS.NS', 'TTKPRESTIG.NS', 'TV18BRDCST.NS', 'TVSMOTOR.NS', 'TANLA.NS', 'TASTYBITE.NS', 'TATACHEM.NS',  'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMTRDVR.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TEAMLEASE.NS', 'TECHM.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'THYROCARE.NS', 'TIMKEN.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UFLEX.NS', 'UPL.NS', 'UTIAMC.NS',  'UJJIVANSFB.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS',  'VGUARD.NS', 'VMART.NS', 'VIPIND.NS', 'VSTIND.NS', 'VAIBHAVGBL.NS', 'VAKRANGEE.NS', 'VALIANTORG.NS', 'VTL.NS', 'VARROC.NS', 'VBL.NS', 'VEDL.NS', 'VENKEYS.NS', 'VINATIORGA.NS', 'IDEA.NS', 'VOLTAS.NS',  'WELCORP.NS',  'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZYDUSWELL.NS', 'ECLERX.NS']
Scrip_MnPoly = ['HINDUNILVR.NS', 'ASIANPAINT.NS', 'MARUTI.NS', 'EICHERMOT.NS', 'PIDILITIND.NS', 'NESTLEIND.NS', 'GMMPFAUDLR.NS', 'PAGEIND.NS', 'JUBLFOOD.NS', 'TITAN.NS', 'CAMS.NS', 'IEX.NS', 'APLAPOLLO.NS', 'INDIAMART.NS', 'IRCTC.NS', 'CDSL.NS']
scrip_Auto = [ 'APOLLOTYRE.NS','ASAHIINDIA.NS','ASHOKLEY.NS','BAJAJ-AUTO.NS','BALKRISIND.NS','BOSCHLTD.NS','CEATLTD.NS','EICHERMOT.NS','ENDURANCE.NS','ESCORTS.NS','EXIDEIND.NS','HEROMOTOCO.NS','JKTYRE.NS','JTEKTINDIA.NS','JAMNAAUTO.NS','MRF.NS','M&M.NS','MARUTI.NS','MINDACORP.NS','MOTHERSUMI.NS','SUNDRMFAST.NS','SUPRAJIT.NS','TVSMOTOR.NS','TATAMTRDVR.NS','TATAMOTORS.NS','TIINDIA.NS','VARROC.NS','WABCOINDIA.NS']
scrip_Cement = ['ACC.NS','AMBUJACEM.NS','BIRLACORPN.NS','DALBHARAT.NS','GRASIM.NS','HEIDELBERG.NS','INDIACEM.NS','JKCEMENT.NS','JKLAKSHMI.NS','PRSMJOHNSN.NS','SHREECEM.NS','RAMCOCEM.NS','ULTRACEMCO.NS']
scrip_Chem = ['AARTIIND.NS','ALKYLAMINE.NS','ATUL.NS','BASF.NS','BALAMINES.NS','DEEPAKNTR.NS','FINEORG.NS','GALAXYSURF.NS','GUJALKALI.NS','FLUOROCHEM.NS','GNFC.NS','HSCL.NS','LINDEINDIA.NS','NOCIL.NS','NAVINFLUOR.NS','PIDILITIND.NS','RAIN.NS','SRF.NS','SOLARINDS.NS','SUDARSCHEM.NS','TATACHEM.NS','VINATIORGA.NS']
scrip_Cons = ['ASHOKA.NS','BRIGADE.NS','DLF.NS','DBL.NS','ENGINERSIN.NS','GMRINFRA.NS','GODREJPROP.NS','IRB.NS','IRCON.NS','IBREALEST.NS','KNRCON.NS','LT.NS','NBCC.NS','NCC.NS','OBEROIRLTY.NS','PNCINFRA.NS','PHOENIXLTD.NS','PRESTIGE.NS','RVNL.NS','SOBHA.NS','SWSOLAR.NS','SUNTECK.NS']
scrip_ConGD = ['3MINDIA.NS','ADVENZYMES.NS','AKZOINDIA.NS','AMBER.NS','ASIANPAINT.NS','AVANTIFEED.NS','BAJAJCON.NS','BAJAJELEC.NS','BALRAMCHIN.NS','BATAINDIA.NS','BERGEPAINT.NS','BLUESTARCO.NS','BBTC.NS','BRITANNIA.NS','CCL.NS','CENTURYPLY.NS','CERA.NS','COLPAL.NS','CROMPTON.NS','DCMSHRIRAM.NS','DABUR.NS','DIXON.NS','EIDPARRY.NS','EMAMILTD.NS','GILLETTE.NS','GODFRYPHLP.NS','GODREJAGRO.NS','GODREJCP.NS','GODREJIND.NS','GAEL.NS','HATSUN.NS','HAVELLS.NS','HINDUNILVR.NS','IFBIND.NS','ITC.NS','JCHAC.NS','JYOTHYLAB.NS','KRBL.NS','KAJARIACER.NS','KSCL.NS','LAOPALA.NS','MARICO.NS','NESTLEIND.NS','ORIENTELEC.NS','PGHH.NS','RADICO.NS','RAJESHEXPO.NS','RELAXO.NS','RESPONIND.NS','SFL.NS','SYMPHONY.NS','TTKPRESTIG.NS','TASTYBITE.NS','TATACONSUM.NS','TITAN.NS','UBL.NS','VGUARD.NS','VIPIND.NS','VSTIND.NS','VBL.NS','VENKEYS.NS','VOLTAS.NS','WHIRLPOOL.NS','ZYDUSWELL.NS']
scrip_ConSRVC = ['ABFRL.NS','DMART.NS','CHALET.NS','DELTACORP.NS','EIHOTEL.NS','FCONSUMER.NS','FRETAIL.NS','INDIAMART.NS','INDHOTEL.NS','NAUKRI.NS','JUBLFOOD.NS','JUSTDIAL.NS','LEMONTREE.NS','MHRIL.NS','SHOPERSTOP.NS','TRENT.NS','VMART.NS','VAIBHAVGBL.NS','WESTLIFE.NS']
scrip_Fert = ['BAYERCROP.NS','BHARATRAS.NS','CHAMBLFERT.NS','COROMANDEL.NS','DHANUKA.NS','GSFC.NS','NFL.NS','PIIND.NS','RALLIS.NS','RCF.NS','SHARDACROP.NS','SUMICHEM.NS','UPL.NS']
scrip_Finan = ['AUBANK.NS','AAVAS.NS','ABCAPITAL.NS','AXISBANK.NS','BSE.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BAJAJHLDNG.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'CRISIL.NS', 'CSBBANK.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CGCL.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CUB.NS', 'CREDITACC.NS', 'DCBBANK.NS', 'DHANI.NS', 'EDELWEISS.NS', 'FEDERALBNK.NS', 'GICRE.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HUDCO.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'ISEC.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IDFC.NS', 'IIFL.NS', 'IBULHSGFIN.NS', 'INDIANB.NS', 'IEX.NS', 'IOB.NS', 'INDUSINDBK.NS', 'JMFINANCIL.NS', 'KARURVYSYA.NS', 'KOTAKBANK.NS',  'LICHSGFIN.NS', 'MASFIN.NS', 'MAHSCOOTER.NS', 'M&MFIN.NS', 'MANAPPURAM.NS', 'MFSL.NS', 'MOTILALOFS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'PNBHOUSING.NS', 'PEL.NS', 'PFC.NS', 'PNB.NS', 'RBLBANK.NS', 'RECLTD.NS', 'SBICARD.NS', 'SBILIFE.NS',   'SPANDANA.NS', 'SBIN.NS', 'SUNDARMFIN.NS', 'TATAINVEST.NS', 'NIACL.NS', 'UCOBANK.NS',  'UJJIVANSFB.NS', 'UNIONBANK.NS', 'YESBANK.NS']
scrip_Health = ['APOLLOHOSP.NS', 'ASTERDM.NS', 'LALPATHLAB.NS', 'FORTIS.NS', 'METROPOLIS.NS', 'NH.NS', 'POLYMED.NS', 'SYNGENE.NS', 'THYROCARE.NS']
scrip_Manufc = ['ABB.NS', 'POWERINDIA.NS', 'AIAENG.NS', 'ASTRAL.NS', 'BEML.NS', 'BALMLAWRIE.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'CARBORUNIV.NS', 'COCHINSHIP.NS', 'CUMMINSIND.NS', 'EPL.NS', 'ELGIEQUIP.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'GEPIL.NS', 'GMMPFAUDLR.NS', 'GRSE.NS', 'GRAPHITE.NS', 'GREAVESCOT.NS', 'GRINDWELL.NS', 'HEG.NS', 'HAL.NS', 'HONAUT.NS', 'HUHTAMAKI.NS', 'INGERRAND.NS', 'KEI.NS', 'KSB.NS', 'LAXMIMACH.NS',  'NILKAMAL.NS', 'ORIENTREF.NS', 'POLYCAB.NS', 'POLYPLEX.NS', 'PRINCEPIPE.NS', 'SKFINDIA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SIEMENS.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TRITURBINE.NS', 'UFLEX.NS']
scrip_IT = ['AFFLE.NS', 'BSOFT.NS', 'COFORGE.NS', 'CYIENT.NS', 'FSL.NS', 'HCLTECH.NS', 'INFIBEAM.NS', 'INFY.NS', 'INTELLECT.NS', 'KPITTECH.NS', 'LTTS.NS',   'MPHASIS.NS', 'OFSS.NS', 'PERSISTENT.NS', 'SONATSOFTW.NS', 'TANLA.NS', 'TCS.NS', 'TATAELXSI.NS', 'TECHM.NS', 'VAKRANGEE.NS', 'WIPRO.NS', 'ZENSARTECH.NS', 'ECLERX.NS']
scrip_Metal = ['APLAPOLLO.NS', 'ADANIENT.NS', 'COALINDIA.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDZINC.NS', 'JSWSTEEL.NS', 'JINDALSAW.NS',   'JINDALSTEL.NS', 'MOIL.NS', 'MAHSEAMLES.NS', 'MIDHANI.NS', 'NMDC.NS', 'NATIONALUM.NS', 'RATNAMANI.NS', 'SAIL.NS', 'TATASTEEL.NS', 'VEDL.NS', 'WELCORP.NS']
scrip_Oil = ['BPCL.NS', 'CASTROLIND.NS', 'GAIL.NS', 'GUJGASLTD.NS', 'GSPL.NS', 'GULFOILLUB.NS', 'HINDPETRO.NS', 'IOC.NS', 'IGL.NS', 'MGL.NS', 'MRPL.NS', 'ONGC.NS', 'OIL.NS', 'PETRONET.NS', 'RELIANCE.NS']
scrip_Pharm = ['AARTIDRUGS.NS', 'ABBOTINDIA.NS', 'AJANTPHARM.NS', 'ALEMBICLTD.NS', 'APLLTD.NS', 'ALKEM.NS', 'ASTRAZEN.NS', 'AUROPHARMA.NS', 'BIOCON.NS', 'BLISSGVS.NS', 'CAPLIPOINT.NS', 'CIPLA.NS', 'DCAL.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'ERIS.NS', 'FDC.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GRANULES.NS', 'IOLCP.NS', 'INDOCO.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'LAURUSLABS.NS', 'LUPIN.NS', 'NATCOPHARM.NS', 'PFIZER.NS', 'PGHL.NS', 'SANOFI.NS', 'SEQUENT.NS', 'SHILPAMED.NS', 'SOLARA.NS', 'STAR.NS', 'SPARC.NS', 'SUNPHARMA.NS', 'SUVENPHAR.NS', 'TORNTPHARM.NS', 'WOCKPHARMA.NS']
scrip_Powr = ['ADANIGREEN.NS', 'CESC.NS', 'JSWENERGY.NS',  'KEC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NTPC.NS', 'POWERGRID.NS', 'SJVN.NS', 'TATAPOWER.NS', 'TORNTPOWER.NS']
scrip_Serv = ['ADANIPORTS.NS', 'BLUEDART.NS', 'CONCOR.NS', 'GESHIP.NS', 'GPPL.NS', 'IRCTC.NS', 'INDIGO.NS', 'MMTC.NS', 'MAHLOG.NS', 'NESCO.NS', 'QUESS.NS', 'RITES.NS', 'REDINGTON.NS', 'SIS.NS', 'SCI.NS',  'TCIEXP.NS', 'TEAMLEASE.NS']
scrip_Tele = ['BHARTIARTL.NS', 'HFCL.NS', 'ITI.NS', 'INDUSTOWER.NS', 'STLTECH.NS', 'TATACOMM.NS', 'IDEA.NS']
scrip_Text = ['ALOKINDS.NS', 'GARFIBRES.NS', 'ICIL.NS', 'KPRMILL.NS', 'LUXIND.NS', 'PAGEIND.NS', 'RAYMOND.NS', 'SWANENERGY.NS', 'TCNSBRANDS.NS', 'TRIDENT.NS', 'VTL.NS', 'WELSPUNIND.NS']
scrip_All = ['20MICRONS.NS', '21STCENMGM.NS', '3IINFOLTD.NS', '3MINDIA.NS', '3PLAND.NS', '4THDIM.NS', '5PAISA.NS', 'A2ZINFRA.NS', 'AAATECH.NS', 'AAKASH.NS', 'AAREYDRUGS.NS', 'AARON.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AARTISURF.NS', 'AARVEEDEN.NS', 'AARVI.NS', 'AAVAS.NS', 'ABAN.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABCOTS.NS', 'ABFRL.NS', 'ABMINTLLTD.NS', 'ABSLAMC.NS', 'ABSLBANETF.NS', 'ABSLNN50ET.NS', 'ACC.NS', 'ACCELYA.NS', 'ACCURACY.NS', 'ACE.NS', 'ACEINTEG.NS', 'ACI.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ADFFOODS.NS', 'ADL.NS', 'ADORWELD.NS', 'ADROITINFO.NS', 'ADSL.NS', 'ADVANIHOTR.NS', 'ADVENZYMES.NS', 'AETHER.NS', 'AFFLE.NS', 'AGARIND.NS', 'AGI.NS', 'AGNI.NS', 'AGRITECH.NS', 'AGROPHOS.NS', 'AGSTRA.NS', 'AHLADA.NS', 'AHLEAST.NS', 'AHLUCONT.NS', 'AIAENG.NS', 'AILIMITED.NS', 'AIRAN.NS', 'AIROLAM.NS', 'AIRTELPP.NS', 'AISL.NS', 'AJANTPHARM.NS', 'AJMERA.NS', 'AJOONI.NS', 'AJOONI-RE.NS', 'AJRINFRA.NS', 'AKASH.NS', 'AKG.NS', 'AKSHAR.NS', 'AKSHARCHEM.NS', 'AKSHOPTFBR.NS', 'AKZOINDIA.NS', 'ALANKIT.NS', 'ALBERTDAVD.NS', 'ALEMBICLTD.NS', 'ALICON.NS', 'ALKALI.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALLCARGO.NS', 'ALLSEC.NS', 'ALMONDZ.NS', 'ALOKINDS.NS', 'ALPA.NS', 'ALPHAGEO.NS', 'ALPSINDUS.NS', 'AMBER.NS', 'AMBICAAGAR.NS', 'AMBIKCO.NS', 'AMBUJACEM.NS', 'AMDIND.NS', 'AMEYA.NS', 'AMIABLE.NS', 'AMIORG.NS', 'AMJLAND.NS', 'AMJUMBO.NS', 'AMRUTANJAN.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANDHRAPAP.NS', 'ANDHRSUGAR.NS', 'ANDREWYU.NS', 'ANGELONE.NS', 'ANIKINDS.NS', 'ANKITMETAL.NS', 'ANMOL.NS', 'ANNAPURNA.NS', 'ANSALAPI.NS', 'ANTGRAPHIC.NS', 'ANUP.NS', 'ANURAS.NS', 'APARINDS.NS', 'APCL.NS', 'APCOTEXIND.NS', 'APEX.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLO.NS', 'APOLLOHOSP.NS', 'APOLLOPIPE.NS', 'APOLLOTYRE.NS', 'APOLSINHOT.NS', 'APTECHT.NS', 'APTUS.NS', 'ARCHIDPLY.NS', 'ARCHIES.NS', 'ARENTERP.NS', 'ARIES.NS', 'ARIHANTCAP.NS', 'ARIHANTSUP.NS', 'ARMANFIN.NS', 'AROGRANITE.NS', 'ARROWGREEN.NS', 'ARSHIYA.NS', 'ARSSINFRA.NS', 'ARTEMISMED.NS', 'ARTNIRMAN.NS', 'ARVEE.NS', 'ARVIND.NS', 'ARVINDFASN.NS', 'ARVSMART.NS', 'ASAHIINDIA.NS', 'ASAHISONG.NS', 'ASAL.NS', 'ASALCBR.NS', 'ASHAPURMIN.NS', 'ASHIANA.NS', 'ASHIMASYN.NS', 'ASHOKA.NS', 'ASHOKLEY.NS', 'ASIANENE.NS', 'ASIANHOTNR.NS', 'ASIANPAINT.NS', 'ASIANTILES.NS', 'ASPINWALL.NS', 'ASTEC.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAMICRO.NS', 'ASTRAZEN.NS', 'ASTRON.NS', 'ATALREAL.NS', 'ATFL.NS', 'ATGL.NS', 'ATLANTA.NS', 'ATUL.NS', 'ATULAUTO.NS', 'AUBANK.NS', 'AURDIS.NS', 'AURIONPRO.NS', 'AUROPHARMA.NS', 'AURUM.NS', 'AURUMPP.NS', 'AUSOMENT.NS', 'AUTOAXLES.NS', 'AUTOBEES.NS', 'AUTOIND.NS', 'AVADHSUGAR.NS', 'AVANTIFEED.NS', 'AVG.NS', 'AVROIND.NS', 'AVTNPL.NS', 'AWHCL.NS', 'AWL.NS', 'AXISBANK.NS', 'AXISBNKETF.NS', 'AXISBPSETF.NS', 'AXISCADES.NS', 'AXISCETF.NS', 'AXISGOLD.NS', 'AXISHCETF.NS', 'AXISILVER.NS', 'AXISNIFTY.NS', 'AXISTECETF.NS', 'AXITA.NS', 'AYMSYNTEX.NS',
'BAFNAPH.NS', 'BAGFILMS.NS', 'BAHETI.NS', 'BAJAJ-AUTO.NS', 'BAJAJCON.NS', 'BAJAJELEC.NS', 'BAJAJFINSV.NS', 'BAJAJHCARE.NS', 'BAJAJHIND.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALAJITELE.NS', 'BALAMINES.NS', 'BALAXI.NS', 'BALKRISHNA.NS', 'BALKRISIND.NS', 'BALLARPUR.NS', 'BALMLAWRIE.NS', 'BALPHARMA.NS', 'BALRAMCHIN.NS', 'BANARBEADS.NS', 'BANARISUG.NS', 'BANCOINDIA.NS', 'BANDHANBNK.NS', 'BANG.NS', 'BANKA.NS', 'BANKBARODA.NS', 'BANKBEES.NS', 'BANKINDIA.NS', 'BANSWRAS.NS', 'BARBEQUE.NS', 'BASF.NS', 'BASML.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBETF0432.NS', 'BBL.NS', 'BBOX.NS', 'BBTC.NS', 'BBTCL.NS', 'BCG.NS', 'BCLIND.NS', 'BCONCEPTS.NS', 'BCP.NS', 'BDL.NS', 'BEARDSELL.NS', 'BECTORFOOD.NS', 'BEDMUTHA.NS', 'BEL.NS', 'BEML.NS', 'BEPL.NS', 'BERGEPAINT.NS', 'BESTAGRO.NS', 'BETA.NS', 'BEWLTD.NS', 'BFINVEST.NS', 'BFUTILITIE.NS', 'BGRENERGY.NS', 'BHAGCHEM.NS', 'BHAGERIA.NS', 'BHAGYANGR.NS', 'BHAGYAPROP.NS', 'BHANDARI.NS', 'BHARATFORG.NS', 'BHARATGEAR.NS', 'BHARATRAS.NS', 'BHARATWIRE.NS', 'BHARTIARTL.NS', 'BHEL.NS', 'BIGBLOC.NS', 'BIKAJI.NS', 'BIL.NS', 'BINDALAGRO.NS', 'BIOCON.NS', 'BIOFILCHEM.NS', 'BIRET.NS', 'BIRLACABLE.NS', 'BIRLACORPN.NS', 'BIRLAMONEY.NS', 'BIRLATYRE.NS', 'BKMINDST.NS', 'BLBLIMITED.NS', 'BLISSGVS.NS', 'BLKASHYAP.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BMETRICS.NS', 'BODALCHEM.NS', 'BOHRAIND.NS', 'BOMDYEING.NS', 'BOROLTD.NS', 'BORORENEW.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BPL.NS', 'BRIGADE.NS', 'BRIGHT.NS', 'BRITANNIA.NS', 'BRNL.NS', 'BROOKS.NS', 'BSE.NS', 'BSHSL.NS', 'BSL.NS', 'BSLGOLDETF.NS', 'BSLNIFTY.NS', 'BSLSENETFG.NS', 'BSOFT.NS', 'BTML.NS', 'BURNPUR.NS', 'BUTTERFLY.NS', 'BVCL.NS', 'BYKE.NS', 'CADSYS.NS', 'CALSOFT.NS', 'CAMLINFINE.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CANTABIL.NS', 'CAPACITE.NS', 'CAPLIPOINT.NS', 'CAPTRUST.NS', 'CARBORUNIV.NS', 'CAREERP.NS', 'CARERATING.NS', 'CARTRADE.NS', 'CARYSIL.NS', 'CASTROLIND.NS', 'CCHHL.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CELEBRITY.NS', 'CENTENKA.NS', 'CENTEXT.NS', 'CENTRALBK.NS', 'CENTRUM.NS', 'CENTUM.NS', 'CENTURYPLY.NS', 'CENTURYTEX.NS', 'CERA.NS', 'CEREBRAINT.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHEMBOND.NS', 'CHEMCON.NS', 'CHEMFAB.NS', 'CHEMPLASTS.NS', 'CHENNPETRO.NS', 'CHEVIOT.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIGNITITEC.NS', 'CINELINE.NS', 'CINEVISTA.NS', 'CIPLA.NS', 'CLEAN.NS', 'CLEDUCATE.NS', 'CLNINDIA.NS', 'CLOUD.NS', 'CLSEL.NS', 'CMICABLES.NS', 'CMMIPL.NS', 'CMRSL.NS', 'CMSINFO.NS', 'COALINDIA.NS', 'COASTCORP.NS', 'COASTPP.NS', 'COCHINSHIP.NS', 'COFFEEDAY.NS', 'COFORGE.NS', 'COLPAL.NS', 'COMPINFO.NS', 'COMPUSOFT.NS', 'CONCOR.NS', 'CONFIPET.NS', 'CONSOFINVT.NS', 'CONSUMBEES.NS', 'CONTI.NS', 'CONTROLPR.NS', 'COOLCAPS.NS', 'CORALFINAC.NS', 'CORDSCABLE.NS', 'COROMANDEL.NS', 'COSMOFIRST.NS', 'COUNCODOS.NS', 'CPSEETF.NS', 'CRAFTSMAN.NS', 'CREATIVE.NS', 'CREATIVEYE.NS', 'CREDITACC.NS', 'CREST.NS', 'CRISIL.NS', 'CROMPTON.NS',
'CROWN.NS', 'CSBBANK.NS', 'CSLFINANCE.NS', 'CTE.NS', 'CUB.NS', 'CUBEXTUB.NS', 'CUMMINSIND.NS', 'CUPID.NS', 'CYBERMEDIA.NS', 'CYBERTECH.NS', 'CYIENT.NS', 'DAAWAT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DALMIASUG.NS', 'DAMODARIND.NS', 'DANGEE.NS', 'DATAMATICS.NS', 'DATAPATTNS.NS', 'DBCORP.NS', 'DBL.NS', 'DBOL.NS', 'DBREALTY.NS', 'DBSTOCKBRO.NS', 'DCAL.NS', 'DCBBANK.NS', 'DCI.NS', 'DCM.NS', 'DCMFINSERV.NS', 'DCMNVL.NS', 'DCMSHRIRAM.NS', 'DCMSRIND.NS', 'DCW.NS', 'DCXINDIA.NS', 'DECCANCE.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DEEPENR.NS', 'DEEPINDS.NS', 'DELHIVERY.NS', 'DELPHIFX.NS', 'DELTACORP.NS', 'DELTAMAGNT.NS', 'DEN.NS', 'DENORA.NS', 'DEVIT.NS', 'DEVYANI.NS', 'DFMFOODS.NS', 'DGCONTENT.NS', 'DHAMPURSUG.NS', 'DHANBANK.NS', 'DHANI.NS', 'DHANILOANS.NS', 'DHANUKA.NS', 'DHARMAJ.NS', 'DHARSUGAR.NS', 'DHRUV.NS', 'DHUNINV.NS', 'DIAMONDYD.NS', 'DICIND.NS', 'DIGISPICE.NS', 'DIL.NS', 'DISHTV.NS', 'DIVISLAB.NS', 'DIVOPPBEES.NS', 'DIXON.NS', 'DJML.NS', 'DKEGL.NS', 'DLF.NS', 'DLINKINDIA.NS', 'DMART.NS', 'DMCC.NS', 'DNAMEDIA.NS', 'DODLA.NS', 'DOLATALGO.NS', 'DOLLAR.NS', 'DONEAR.NS', 'DPABHUSHAN.NS', 'DPSCLTD.NS', 'DPWIRES.NS', 'DRCSYSTEMS.NS', 'DREAMFOLKS.NS', 'DREDGECORP.NS', 'DRREDDY.NS', 'DSPN50ETF.NS', 'DSPNEWETF.NS', 'DSPQ50ETF.NS', 'DSPSILVETF.NS', 'DSSL.NS', 'DTIL.NS', 'DUCON.NS', 'DUGLOBAL.NS', 'DVL.NS', 'DWARKESH.NS', 'DYCL.NS', 'DYNAMATECH.NS', 'DYNAMIC.NS', 'DYNPRO.NS', 'E2E.NS', 'EASEMYTRIP.NS', 'EASTSILK.NS', 'EBBETF0423.NS', 'EBBETF0425.NS', 'EBBETF0430.NS', 'EBBETF0431.NS', 'EC4RG.NS', 'ECLERX.NS', 'ECLFINANCE.NS', 'EDELWEISS.NS', 'EDUCOMP.NS', 'EHFLNCD.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIFFL.NS', 'EIHAHOTELS.NS', 'EIHOTEL.NS', 'EIMCOELECO.NS', 'EKC.NS', 'ELDEHSG.NS', 'ELECON.NS', 'ELECTCAST.NS', 'ELECTHERM.NS', 'ELGIEQUIP.NS', 'ELGIRUBCO.NS', 'EMAMILTD.NS', 'EMAMIPAP.NS', 'EMAMIREAL.NS', 'EMBASSY.NS', 'EMIL.NS', 'EMKAY.NS', 'EMMBI.NS', 'EMUDHRA.NS', 'ENDURANCE.NS', 'ENERGYDEV.NS', 'ENGINERSIN.NS', 'ENIL.NS', 'EPL.NS', 'EQUIPPP.NS', 'EQUITASBNK.NS', 'ERFLNCDI.NS', 'ERIS.NS', 'EROSMEDIA.NS', 'ESABINDIA.NS', 'ESCORTS.NS', 'ESSARSHPNG.NS', 'ESTER.NS', 'ETHOSLTD.NS', 'EUROBOND.NS', 'EUROTEXIND.NS', 'EVEREADY.NS', 'EVERESTIND.NS', 'EXCEL.NS', 'EXCELINDUS.NS', 'EXIDEIND.NS', 'EXPLEOSOL.NS', 'EXXARO.NS', 'FACT.NS', 'FAIRCHEMOR.NS', 'FAZE3Q.NS', 'FCL.NS', 'FCONSUMER.NS', 'FCSSOFT.NS', 'FDC.NS', 'FEDERALBNK.NS', 'FEL.NS', 'FELDVR.NS', 'FELIX.NS', 'FIBERWEB.NS', 'FIDEL.NS', 'FIEMIND.NS', 'FILATEX.NS', 'FINCABLES.NS', 'FINEORG.NS', 'FINOPB.NS', 'FINPIPE.NS', 'FIVESTAR.NS', 'FLEXITUFF.NS', 'FLFL.NS', 'FLUOROCHEM.NS', 'FMGOETZE.NS', 'FMNL.NS', 'FOCUS.NS', 'FOODSIN.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FOSECOIND.NS', 'FROG.NS', 'FSC.NS', 'FSL.NS', 'FUSION.NS', 'GABRIEL.NS', 'GAEL.NS', 'GAIL.NS', 'GAL.NS', 'GALAXYSURF.NS', 'GALLANTT.NS', 'GANDHITUBE.NS', 'GANECOS.NS', 'GANESHBE.NS',
'GANESHHOUC.NS', 'GANGAFORGE.NS', 'GANGESSECU.NS', 'GARFIBRES.NS', 'GATEWAY.NS', 'GATI.NS', 'GAYAHWS.NS', 'GAYAPROJ.NS', 'GEECEE.NS', 'GEEKAYWIRE.NS', 'GENCON.NS', 'GENESYS.NS', 'GENUSPAPER.NS', 'GENUSPOWER.NS', 'GEOJITFSL.NS', 'GEPIL.NS', 'GESHIP.NS', 'GET&D.NS', 'GFLLIMITED.NS', 'GHCL.NS', 'GICHSGFIN.NS', 'GICL.NS', 'GICRE.NS', 'GILLANDERS.NS', 'GILLETTE.NS', 'GILT5YBEES.NS', 'GINNIFILA.NS', 'GIPCL.NS', 'GIRIRAJ.NS', 'GKWLIMITED.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GLFL.NS', 'GLOBAL.NS', 'GLOBALVECT.NS', 'GLOBE.NS', 'GLOBUSSPR.NS', 'GLS.NS', 'GMBREW.NS', 'GMDCLTD.NS', 'GMMPFAUDLR.NS', 'GMRINFRA.NS', 'GMRP&UI.NS', 'GNA.NS', 'GNFC.NS', 'GOACARBON.NS', 'GOCLCORP.NS', 'GOCOLORS.NS', 'GODFRYPHLP.NS', 'GODHA.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GOENKA.NS', 'GOKEX.NS', 'GOKUL.NS', 'GOKULAGRO.NS', 'GOLDBEES.NS', 'GOLDENTOBC.NS', 'GOLDIAM.NS', 'GOLDSHARE.NS', 'GOLDSTAR.NS', 'GOLDTECH.NS', 'GOODLUCK.NS', 'GOODYEAR.NS', 'GOYALALUM.NS', 'GPIL.NS', 'GPPL.NS', 'GPTINFRA.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAUWEIL.NS', 'GRAVITA.NS', 'GREAVESCOT.NS', 'GREENLAM.NS', 'GREENPANEL.NS', 'GREENPLY.NS', 'GREENPOWER.NS', 'GRINDWELL.NS', 'GRINFRA.NS', 'GRMOVER.NS', 'GROBTEA.NS', 'GRPLTD.NS', 'GRSE.NS', 'GRWRHITECH.NS', 'GSCLCEMENT.NS', 'GSFC.NS', 'GSPL.NS', 'GSS.NS', 'GSTL.NS', 'GTL.NS', 'GTLINFRA.NS', 'GTPL.NS', 'GUFICBIO.NS', 'GUJALKALI.NS', 'GUJAPOLLO.NS', 'GUJGASLTD.NS', 'GUJRAFFIA.NS', 'GULFOILLUB.NS', 'GULFPETRO.NS', 'GULPOLY.NS', 'GVKPIL.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HARDWYN.NS', 'HARIOMPIPE.NS', 'HARRMALAYA.NS', 'HARSHA.NS', 'HATHWAY.NS', 'HATSUN.NS', 'HAVELLS.NS', 'HAVISHA.NS', 'HBANKETF.NS', 'HBLPOWER.NS', 'HBSL.NS', 'HCC.NS', 'HCG.NS', 'HCL-INSYS.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCGROWTH.NS', 'HDFCLIFE.NS', 'HDFCLOWVOL.NS', 'HDFCMFGETF.NS', 'HDFCMOMENT.NS', 'HDFCNEXT50.NS', 'HDFCNIF100.NS', 'HDFCNIFETF.NS', 'HDFCNIFIT.NS', 'HDFCPVTBAN.NS', 'HDFCQUAL.NS', 'HDFCSENETF.NS', 'HDFCSILVER.NS', 'HDFCVALUE.NS', 'HDIL.NS', 'HEADSUP.NS', 'HEALTHY.NS', 'HECPROJECT.NS', 'HEG.NS', 'HEIDELBERG.NS', 'HEMIPROP.NS', 'HERANBA.NS', 'HERCULES.NS', 'HERITGFOOD.NS', 'HEROMOTOCO.NS', 'HESTERBIO.NS', 'HEXATRADEX.NS', 'HFCL.NS', 'HGINFRA.NS', 'HGS.NS', 'HIKAL.NS', 'HIL.NS', 'HILTON.NS', 'HIMATSEIDE.NS', 'HINDALCO.NS', 'HINDCOMPOS.NS', 'HINDCON.NS', 'HINDCOPPER.NS', 'HINDMOTORS.NS', 'HINDOILEXP.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDWAREAP.NS', 'HINDZINC.NS', 'HIRECT.NS', 'HISARMETAL.NS', 'HITECH.NS', 'HITECHCORP.NS', 'HITECHGEAR.NS', 'HLEGLAS.NS', 'HLVLTD.NS', 'HMT.NS', 'HMVL.NS', 'HNDFDS.NS', 'HNGSNGBEES.NS', 'HOMEFIRST.NS', 'HONAUT.NS', 'HONDAPOWER.NS', 'HOVS.NS', 'HPAL.NS', 'HPIL.NS', 'HPL.NS', 'HSCL.NS', 'HTMEDIA.NS', 'HUBTOWN.NS', 'HUDCO.NS', 'HUHTAMAKI.NS', 'HYBRIDFIN.NS', 'IBMFNIFTY.NS', 'IBREALEST.NS', 'IBULHSGFIN.NS', 'ICDSLTD.NS', 'ICEMAKE.NS', 'ICICI500.NS', 'ICICI5GSEC.NS',
'ICICIALPLV.NS', 'ICICIAUTO.NS', 'ICICIB22.NS', 'ICICIBANK.NS', 'ICICIBANKN.NS', 'ICICIBANKP.NS', 'ICICICONSU.NS', 'ICICIFIN.NS', 'ICICIFMCG.NS', 'ICICIGI.NS', 'ICICIGOLD.NS', 'ICICIINFRA.NS', 'ICICILIQ.NS', 'ICICILOVOL.NS', 'ICICIM150.NS', 'ICICIMCAP.NS', 'ICICIMOM30.NS', 'ICICINF100.NS', 'ICICINIFTY.NS', 'ICICINV20.NS', 'ICICINXT50.NS', 'ICICIPHARM.NS', 'ICICIPRULI.NS', 'ICICISENSX.NS', 'ICICISILVE.NS', 'ICICITECH.NS', 'ICIL.NS', 'ICRA.NS', 'IDBI.NS', 'IDBIGOLD.NS', 'IDEA.NS', 'IDFC.NS', 'IDFCFIRSTB.NS', 'IDFNIFTYET.NS', 'IEX.NS', 'IFBAGRO.NS', 'IFBIND.NS', 'IFCI.NS', 'IFGLEXPOR.NS', 'IGARASHI.NS', 'IGL.NS', 'IGPL.NS', 'IIFCL.NS', 'IIFL.NS', 'IIFLSEC.NS', 'IIHFL.NS', 'IITL.NS', 'IL&FSENGG.NS', 'IL&FSTRANS.NS', 'IMAGICAA.NS', 'IMFA.NS', 'IMPAL.NS', 'INCREDFIN.NS', 'INCREDIBLE.NS', 'INDBANK.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAGLYCO.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIANCARD.NS', 'INDIANHUME.NS', 'INDIGO.NS', 'INDIGOPNTS.NS', 'INDIGRID.NS', 'INDLMETER.NS', 'INDNIPPON.NS', 'INDOAMIN.NS', 'INDOBORAX.NS', 'INDOCO.NS', 'INDORAMA.NS', 'INDOSTAR.NS', 'INDOTECH.NS', 'INDOTHAI.NS', 'INDOWIND.NS', 'INDRAMEDCO.NS', 'INDSWFTLAB.NS', 'INDSWFTLTD.NS', 'INDTERRAIN.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INEOSSTYRO.NS', 'INFIBEAM.NS', 'INFOBEAN.NS', 'INFOMEDIA.NS', 'INFRABEES.NS', 'INFY.NS', 'INGERRAND.NS', 'INNOVANA.NS', 'INNOVATIVE.NS', 'INOXGREEN.NS', 'INOXWIND.NS', 'INSECTICID.NS', 'INSPIRISYS.NS', 'INTELLECT.NS', 'INTENTECH.NS', 'INTLCONV.NS', 'INVENTURE.NS', 'IOB.NS', 'IOC.NS', 'IOLCP.NS', 'IONEXCHANG.NS', 'IPCALAB.NS', 'IPL.NS', 'IPSL.NS', 'IRB.NS', 'IRBINVIT.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'IRIS.NS', 'IRISDOREME.NS', 'ISEC.NS', 'ISFT.NS', 'ISGEC.NS', 'ISHAN.NS', 'ISMTLTD.NS', 'ITBEES.NS', 'ITC.NS', 'ITDC.NS', 'ITDCEM.NS', 'ITI.NS', 'IVC.NS', 'IVP.NS', 'IVZINGOLD.NS', 'IVZINNIFTY.NS', 'IWEL.NS', 'IZMO.NS', 'J&KBANK.NS', 'JAGRAN.NS', 'JAGSNPHARM.NS', 'JAIBALAJI.NS', 'JAICORPLTD.NS', 'JAINAM.NS', 'JAIPURKURT.NS', 'JALAN.NS', 'JAMNAAUTO.NS', 'JASH.NS', 'JAYAGROGN.NS', 'JAYBARMARU.NS', 'JAYNECOIND.NS', 'JAYSREETEA.NS', 'JBCHEPHARM.NS', 'JBFIND.NS', 'JBMA.NS', 'JCHAC.NS', 'JETAIRWAYS.NS', 'JETFREIGHT.NS', 'JFLLIFE.NS', 'JHS.NS', 'JINDALPHOT.NS', 'JINDALPOLY.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JINDRILL.NS', 'JINDWORLD.NS', 'JISLDVREQS.NS', 'JISLJALEQS.NS', 'JITFINFRA.NS', 'JKCEMENT.NS', 'JKIL.NS', 'JKLAKSHMI.NS', 'JKPAPER.NS', 'JKTYRE.NS', 'JMA.NS', 'JMCPROJECT.NS', 'JMFINANCIL.NS', 'JOCIL.NS', 'JPASSOCIAT.NS', 'JPINFRATEC.NS', 'JPOLYINVST.NS', 'JPPOWER.NS',   'JSLL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWISPL.NS', 'JSWSTEEL.NS', 'JTEKTINDIA.NS', 'JTLIND.NS', 'JUBLFOOD.NS', 'JUBLINDS.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JUNIORBEES.NS', 'JUSTDIAL.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTISTRUC.NS', 'KABRAEXTRU.NS', 'KAJARIACER.NS', 'KAKATCEM.NS',  'KALYANIFRG.NS', 'KALYANKJIL.NS', 'KAMATHOTEL.NS',
'KAMDHENU.NS', 'KANANIIND.NS', 'KANDARP.NS', 'KANORICHEM.NS', 'KANPRPLA.NS',  'KAPSTON.NS', 'KARMAENG.NS', 'KARURVYSYA.NS', 'KAUSHALYA.NS', 'KAVVERITEL.NS', 'KAYA.NS', 'KAYNES.NS', 'KBCGLOBAL.NS', 'KCP.NS', 'KCPSUGIND.NS', 'KDDL.NS', 'KEC.NS', 'KECL.NS', 'KEEPLEARN.NS', 'KEI.NS', 'KELLTONTEC.NS', 'KENNAMET.NS', 'KERNEX.NS', 'KESORAMIND.NS', 'KEYFINSERV.NS', 'KHADIM.NS', 'KHAICHEM.NS', 'KHAITANLTD.NS', 'KHANDSE.NS', 'KHFM.NS', 'KICL.NS', 'KILITCH.NS', 'KIMS.NS', 'KINGFA.NS', 'KIOCL.NS', 'KIRIINDUS.NS', 'KIRLFER.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KIRLOSIND.NS', 'KITEX.NS', 'KKCL.NS', 'KKVAPOW.NS', 'KMSUGAR.NS', 'KNAGRI.NS', 'KNRCON.NS', 'KOHINOOR.NS', 'KOKUYOCMLN.NS', 'KOLTEPATIL.NS', 'KOPRAN.NS', 'KORE.NS', 'KOTAKALPHA.NS', 'KOTAKBANK.NS', 'KOTAKBKETF.NS', 'KOTAKCONS.NS', 'KOTAKGOLD.NS', 'KOTAKIT.NS', 'KOTAKLOVOL.NS', 'KOTAKMID50.NS', 'KOTAKMNC.NS', 'KOTAKNIFTY.NS', 'KOTAKNV20.NS', 'KOTAKPSUBK.NS', 'KOTARISUG.NS', 'KOTHARIPET.NS', 'KOTHARIPRO.NS', 'KOTYARK.NS', 'KOVAI.NS', 'KPIGREEN.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KRBL.NS', 'KREBSBIO.NS', 'KRIDHANINF.NS', 'KRISHANA.NS', 'KRISHIVAL.NS', 'KRISHNADEF.NS', 'KRITI.NS', 'KRITIKA.NS', 'KRITINUT.NS', 'KRSNAA.NS', 'KSB.NS', 'KSCL.NS', 'KSHITIJPOL.NS', 'KSL.NS', 'KSOLVES.NS', 'KTKBANK.NS', 'KUANTUM.NS',  'L&TFINANCE.NS', 'LAGNAM.NS', 'LAKPRE.NS', 'LALPATHLAB.NS', 'LAMBODHARA.NS', 'LANCER.NS', 'LAOPALA.NS', 'LASA.NS', 'LATENTVIEW.NS', 'LATTEYS.NS', 'LAURUSLABS.NS', 'LAXMICOT.NS', 'LAXMIMACH.NS', 'LCCINFOTEC.NS', 'LEMERITE.NS', 'LEMONTREE.NS', 'LEXUS.NS', 'LFIC.NS', 'LGBBROSLTD.NS', 'LGBFORGE.NS', 'LIBAS.NS', 'LIBERTSHOE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LICNETFGSC.NS', 'LICNETFN50.NS', 'LICNETFSEN.NS', 'LICNFNHGP.NS', 'LIKHITHA.NS', 'LINC.NS', 'LINCOLN.NS', 'LINDEINDIA.NS', 'LIQUIDBEES.NS', 'LIQUIDETF.NS', 'LLOYDS.NS', 'LODHA.NS', 'LOKESHMACH.NS', 'LOTUSEYE.NS', 'LOVABLE.NS', 'LOYALTEX.NS', 'LPDC.NS', 'LSIL.NS', 'LT.NS', 'LTGILTBEES.NS', 'LTIM.NS', 'LTTS.NS', 'LUMAXIND.NS', 'LUMAXTECH.NS', 'LUPIN.NS', 'LUXIND.NS', 'LXCHEM.NS', 'LYKALABS.NS', 'LYPSAGEMS.NS', 'M&M.NS', 'M&MFIN.NS', 'MAANALU.NS', 'MACPOWER.NS', 'MADHAV.NS', 'MADHAVBAUG.NS', 'MADHUCON.NS', 'MADRASFERT.NS', 'MAESGETF.NS', 'MAFANG.NS', 'MAFSETF.NS', 'MAGADSUGAR.NS', 'MAGNUM.NS', 'MAHABANK.NS', 'MAHAPEXLTD.NS', 'MAHASTEEL.NS', 'MAHEPC.NS', 'MAHESHWARI.NS', 'MAHICKRA.NS',  'MAHKTECH.NS', 'MAHLIFE.NS', 'MAHLOG.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MAITHANALL.NS', 'MAKS.NS', 'MALLCOM.NS', 'MALUPAPER.NS', 'MAM150ETF.NS', 'MAMFGETF.NS', 'MAN50ETF.NS', 'MANAKALUCO.NS', 'MANAKCOAT.NS', 'MANAKSIA.NS', 'MANAKSTEEL.NS', 'MANALIPETC.NS', 'MANAPPURAM.NS', 'MANGALAM.NS', 'MANGCHEFER.NS', 'MANGLMCEM.NS', 'MANINDS.NS', 'MANINFRA.NS', 'MANORAMA.NS', 'MANORG.NS', 'MANUGRAPH.NS', 'MANXT50.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARALOVER.NS', 'MARATHON.NS', 'MARICO.NS', 'MARINE.NS', 'MARKSANS.NS', 'MARSHALL.NS', 'MARUTI.NS', 'MASFIN.NS',
'MASKINVEST.NS', 'MASPTOP50.NS', 'MASTEK.NS', 'MATRIMONY.NS', 'MAWANASUG.NS', 'MAXHEALTH.NS', 'MAXIND.NS', 'MAXVIL.NS', 'MAYURUNIQ.NS', 'MAZDA.NS', 'MAZDOCK.NS', 'MBAPL.NS', 'MBECL.NS', 'MBLINFRA.NS', 'MC1RG.NS',  'MCL.NS', 'MCLEODRUSS.NS', 'MCX.NS', 'MEDANTA.NS', 'MEDICAMEQ.NS', 'MEDICO.NS', 'MEDPLUS.NS', 'MEGAFLEX.NS', 'MEGASOFT.NS', 'MEGASTAR.NS', 'MENONBE.NS', 'MEP.NS', 'MERCATOR.NS', 'METALFORGE.NS', 'METROBRAND.NS', 'METROPOLIS.NS', 'MFL.NS', 'MFSL.NS', 'MGEL.NS', 'MGL.NS', 'MHHL.NS', 'MHLXMIRU.NS', 'MHRIL.NS', 'MICEL.NS', 'MID150BEES.NS', 'MIDHANI.NS', 'MILTON.NS', 'MINDACORP.NS', 'MINDPOOL.NS', 'MINDSPACE.NS', 'MINDTECK.NS', 'MIRCELECTR.NS', 'MIRZAINT.NS', 'MITCON.NS', 'MITTAL.NS', 'MKPL.NS', 'MMFL.NS', 'MMP.NS', 'MMTC.NS', 'MODIRUBBER.NS', 'MODISONLTD.NS', 'MOGSEC.NS', 'MOHEALTH.NS', 'MOHITIND.NS', 'MOIL.NS', 'MOKSH.NS', 'MOL.NS', 'MOLDTECH.NS', 'MOLDTKPAC.NS', 'MOLOWVOL.NS', 'MOM100.NS', 'MOM50.NS', 'MOMENTUM.NS', 'MOMOMENTUM.NS', 'MON100.NS', 'MONARCH.NS', 'MONQ50.NS', 'MONTECARLO.NS', 'MOQUALITY.NS', 'MORARJEE.NS', 'MOREPENLAB.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MOTOGENFIN.NS', 'MOVALUE.NS', 'MPHASIS.NS', 'MPSLTD.NS', 'MPTODAY.NS', 'MRF.NS', 'MRO-TEK.NS', 'MRPL.NS', 'MSPL.NS', 'MSTCLTD.NS', 'MSUMI.NS', 'MTARTECH.NS', 'MTEDUCARE.NS', 'MTNL.NS', 'MUKANDLTD.NS', 'MUKTAARTS.NS', 'MUNJALAU.NS', 'MUNJALSHOW.NS', 'MURUDCERA.NS', 'MUTHOOTCAP.NS', 'MUTHOOTFIN.NS', 'NABARD.NS', 'NACLIND.NS', 'NAGAFERT.NS', 'NAGREEKCAP.NS', 'NAGREEKEXP.NS', 'NAHARCAP.NS', 'NAHARINDUS.NS', 'NAHARPOLY.NS', 'NAHARSPING.NS', 'NAM-INDIA.NS', 'NARMADA.NS', 'NATCOPHARM.NS', 'NATHBIOGEN.NS', 'NATIONALUM.NS', 'NATNLSTEEL.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVIFIN.NS', 'NAVINFLUOR.NS', 'NAVKARCORP.NS', 'NAVNETEDUL.NS', 'NAZARA.NS', 'NBCC.NS', 'NBIFIN.NS', 'NCC.NS', 'NCLIND.NS', 'NDGL.NS', 'NDL.NS', 'NDRAUTO.NS', 'NDTV.NS', 'NECCLTD.NS', 'NECLIFE.NS', 'NELCAST.NS', 'NELCO.NS', 'NEOGEN.NS', 'NESCO.NS', 'NESTLEIND.NS', 'NETF.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NEXTMEDIA.NS', 'NFL.NS', 'NGIL.NS', 'NGLFINE.NS', 'NH.NS', 'NHAI.NS', 'NHBTF2014.NS', 'NHIT.NS', 'NHPC.NS', 'NIACL.NS', 'NIBL.NS', 'NIDAN.NS', 'NIF100BEES.NS', 'NIFTYBEES.NS', 'NIFTYQLITY.NS', 'NIITLTD.NS', 'NILAINFRA.NS', 'NILASPACES.NS', 'NILKAMAL.NS', 'NIPPOBATRY.NS', 'NIRAJ.NS', 'NITCO.NS', 'NITINSPIN.NS', 'NITIRAJ.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NOCIL.NS', 'NOIDATOLL.NS', 'NORBTEAEXP.NS', 'NOVARTIND.NS', 'NPBET.NS', 'NRAIL.NS', 'NRBBEARING.NS', 'NRL.NS', 'NSIL.NS', 'NTPC.NS', 'NUCLEUS.NS', 'NURECA.NS', 'NUVOCO.NS', 'NV20BEES.NS', 'NXTDIGITAL.NS', 'NYKAA.NS', 'OAL.NS', 'OBCL.NS', 'OBEROIRLTY.NS', 'OCCL.NS', 'OFSS.NS', 'OIL.NS', 'OILCOUNTUB.NS', 'OLECTRA.NS', 'OMAXAUTO.NS', 'OMAXE.NS', 'OMFURN.NS', 'OMINFRAL.NS', 'OMKARCHEM.NS', 'ONELIFECAP.NS', 'ONEPOINT.NS', 'ONGC.NS', 'ONMOBILE.NS',
'ONWARDTEC.NS', 'OPTIEMUS.NS', 'ORBTEXP.NS', 'ORCHPHARMA.NS', 'ORICONENT.NS', 'ORIENTABRA.NS', 'ORIENTALTL.NS', 'ORIENTBELL.NS', 'ORIENTCEM.NS', 'ORIENTELEC.NS', 'ORIENTHOT.NS', 'ORIENTLTD.NS', 'ORIENTPPR.NS', 'ORISSAMINE.NS', 'ORTEL.NS', 'ORTINLAB.NS', 'OSIAHYPER.NS', 'OSWALAGRO.NS', 'OSWALSEEDS.NS', 'PAGEIND.NS', 'PAISALO.NS', 'PALASHSECU.NS', 'PALREDTEC.NS', 'PANACEABIO.NS', 'PANACHE.NS', 'PANAMAPET.NS', 'PANSARI.NS', 'PAR.NS', 'PARACABLES.NS', 'PARADEEP.NS', 'PARAGMILK.NS', 'PARAS.NS', 'PARSVNATH.NS', 'PARTYCRUS.NS', 'PASHUPATI.NS', 'PASUPTAC.NS', 'PATANJALI.NS', 'PATELENG.NS', 'PATINTLOG.NS', 'PAYTM.NS', 'PCBL.NS', 'PCHFL.NS', 'PCJEWELLER.NS', 'PDMJEPAPER.NS', 'PDSL.NS', 'PEARLPOLY.NS', 'PEL.NS', 'PENIND.NS', 'PENINLAND.NS', 'PERFECT.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PFOCUS.NS', 'PFS.NS', 'PGEL.NS', 'PGHH.NS', 'PGHL.NS', 'PGIL.NS', 'PGINVIT.NS', 'PHANTOMFX.NS', 'PHARMABEES.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PILANIINVS.NS', 'PILITA.NS', 'PIONDIST.NS', 'PIONEEREMB.NS', 'PITTIENG.NS', 'PIXTRANS.NS', 'PKTEA.NS', 'PLASTIBLEN.NS', 'PNB.NS', 'PNBGILTS.NS', 'PNBHOUSING.NS', 'PNC.NS', 'PNCINFRA.NS', 'PODDARHOUS.NS', 'PODDARMENT.NS', 'POKARNA.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POLYPLEX.NS', 'PONNIERODE.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'POWERMECH.NS', 'PPAP.NS', 'PPL.NS', 'PPLPHARMA.NS', 'PRAENG.NS', 'PRAJIND.NS', 'PRAKASH.NS', 'PRAKASHSTL.NS', 'PRAXIS.NS', 'PRECAM.NS', 'PRECISION.NS', 'PRECOT.NS', 'PRECWIRE.NS', 'PREMEXPLN.NS', 'PREMIER.NS', 'PREMIERPOL.NS', 'PRESSMN.NS', 'PRESTIGE.NS', 'PRICOLLTD.NS', 'PRIMESECU.NS', 'PRINCEPIPE.NS', 'PRITI.NS', 'PRITIKA.NS', 'PRITIKAUTO.NS', 'PRIVISCL.NS', 'PROLIFE.NS', 'PROPEQUITY.NS', 'PROZONINTU.NS', 'PRSMJOHNSN.NS', 'PRUDENT.NS', 'PSB.NS', 'PSPPROJECT.NS', 'PSUBNKBEES.NS', 'PTC.NS', 'PTL.NS', 'PULZ.NS', 'PUNJABCHEM.NS', 'PURVA.NS', 'PVP.NS',  'QGOLDHALF.NS', 'QMSMEDI.NS', 'QNIFTY.NS', 'QUADPRO.NS', 'QUESS.NS', 'QUICKHEAL.NS', 'RADAAN.NS', 'RADHIKAJWE.NS', 'RADICO.NS', 'RADIOCITY.NS', 'RAILTEL.NS', 'RAIN.NS', 'RAINBOW.NS', 'RAJESHEXPO.NS', 'RAJMET.NS', 'RAJRATAN.NS', 'RAJSREESUG.NS', 'RAJTV.NS', 'RALLIS.NS', 'RAMANEWS.NS', 'RAMAPHO.NS', 'RAMASTEEL.NS', 'RAMCOCEM.NS', 'RAMCOIND.NS', 'RAMCOSYS.NS', 'RAMKY.NS', 'RAMRAT.NS', 'RANASUG.NS', 'RANEENGINE.NS', 'RANEHOLDIN.NS', 'RATEGAIN.NS', 'RATNAMANI.NS', 'RAYMOND.NS', 'RBA.NS', 'RBL.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'REFEX.NS', 'REGENCERAM.NS', 'RELAXO.NS', 'RELCHEMQ.NS', 'RELIANCE.NS', 'RELIGARE.NS', 'RELINFRA.NS', 'REMSONSIND.NS', 'RENUKA.NS', 'REPCOHOME.NS', 'REPL.NS', 'REPRO.NS', 'RESPONIND.NS', 'REVATHI.NS', 'RGL.NS', 'RHFL.NS', 'RHIM.NS', 'RICHA.NS', 'RICOAUTO.NS', 'RIIL.NS', 'RITCO.NS', 'RITES.NS', 'RITEZONE.NS', 'RKDL.NS', 'RKEC.NS', 'RKFORGE.NS', 'RMCL.NS', 'RMDRIP.NS', 'RML.NS', 'RNAVAL.NS', 'ROHLTD.NS',
'ROLEXRINGS.NS', 'ROLLT.NS', 'ROLTA.NS', 'ROML.NS', 'ROSSARI.NS', 'ROSSELLIND.NS', 'ROTO.NS', 'ROUTE.NS', 'RPGLIFE.NS', 'RPOWER.NS', 'RPPINFRA.NS', 'RPPL.NS', 'RPSGVENT.NS', 'RSSOFTWARE.NS', 'RSWM.NS', 'RSYSTEMS.NS', 'RTNINDIA.NS', 'RTNPOWER.NS', 'RUBYMILLS.NS', 'RUCHINFRA.NS', 'RUCHIRA.NS', 'RUPA.NS', 'RUSHIL.NS', 'RUSTOMJEE.NS', 'RVHL.NS', 'RVNL.NS', 'S&SPOWER.NS', 'SABAR.NS', 'SABEVENTS.NS', 'SABTN.NS', 'SADBHAV.NS', 'SADBHIN.NS', 'SAFARI.NS', 'SAGARDEEP.NS', 'SAGCEM.NS', 'SAIL.NS', 'SAKAR.NS', 'SAKHTISUG.NS', 'SAKSOFT.NS', 'SAKUMA.NS', 'SALASAR.NS', 'SALONA.NS', 'SALSTEEL.NS', 'SALZERELEC.NS', 'SAMBHAAV.NS', 'SANDESH.NS', 'SANDHAR.NS', 'SANGAMIND.NS', 'SANGHIIND.NS', 'SANGHVIMOV.NS', 'SANGINITA.NS', 'SANOFI.NS', 'SANSERA.NS', 'SANWARIA.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SARLAPOLY.NS', 'SARVESHWAR.NS', 'SASKEN.NS', 'SASTASUNDR.NS', 'SATIA.NS', 'SATIN.NS', 'SATINDLTD.NS', 'SBC.NS', 'SBCL.NS', 'SBICARD.NS', 'SBIETFCON.NS', 'SBIETFIT.NS', 'SBIETFPB.NS', 'SBIETFQLTY.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCAPDVR.NS', 'SCHAEFFLER.NS', 'SCHAND.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SCPL.NS', 'SDBL.NS', 'SDL24BEES.NS', 'SDL26BEES.NS', 'SEAMECLTD.NS', 'SECL.NS', 'SECURCRED.NS', 'SECURKLOUD.NS', 'SEJALLTD.NS', 'SELAN.NS', 'SELMC.NS', 'SEPC.NS', 'SEPOWER.NS', 'SEQUENT.NS', 'SERVOTECH.NS', 'SESHAPAPER.NS', 'SETCO.NS', 'SETF10GILT.NS', 'SETFGOLD.NS', 'SETFNIF50.NS', 'SETFNIFBK.NS', 'SETFNN50.NS', 'SETUINFRA.NS', 'SFL.NS', 'SGBAPR28I.NS', 'SGBAUG24.NS', 'SGBAUG27.NS', 'SGBAUG28V.NS', 'SGBAUG29V.NS', 'SGBAUG30.NS', 'SGBD29VIII.NS', 'SGBDC27VII.NS', 'SGBDEC25.NS', 'SGBDEC2512.NS', 'SGBDEC2513.NS', 'SGBFEB24.NS', 'SGBFEB27.NS', 'SGBFEB28IX.NS', 'SGBFEB29XI.NS', 'SGBJ28VIII.NS', 'SGBJAN26.NS', 'SGBJAN29IX.NS', 'SGBJAN29X.NS', 'SGBJAN30IX.NS', 'SGBJU29III.NS', 'SGBJUL25.NS', 'SGBJUL27.NS', 'SGBJUL28IV.NS', 'SGBJUL29IV.NS', 'SGBJUN28.NS', 'SGBJUN29II.NS', 'SGBJUN30.NS', 'SGBMAR24.NS', 'SGBMAR25.NS', 'SGBMAR28X.NS', 'SGBMAR30X.NS', 'SGBMAY26.NS', 'SGBMAY28.NS', 'SGBMAY29I.NS', 'SGBMR29XII.NS', 'SGBN28VIII.NS', 'SGBNOV23.NS', 'SGBNOV24.NS', 'SGBNOV25.NS', 'SGBNOV25VI.NS', 'SGBNOV26.NS', 'SGBNV29VII.NS', 'SGBOC28VII.NS', 'SGBOCT25.NS', 'SGBOCT25IV.NS', 'SGBOCT25V.NS', 'SGBOCT26.NS', 'SGBOCT27.NS', 'SGBOCT27VI.NS', 'SGBSEP24.NS', 'SGBSEP28VI.NS', 'SGBSEP29VI.NS', 'SGIL.NS', 'SGL.NS', 'SHAHALLOYS.NS', 'SHAILY.NS', 'SHAKTIPUMP.NS', 'SHALBY.NS', 'SHALPAINTS.NS', 'SHANKARA.NS', 'SHANTI.NS', 'SHANTIGEAR.NS', 'SHARDACROP.NS', 'SHARDAMOTR.NS', 'SHAREINDIA.NS', 'SHARIABEES.NS', 'SHEMAROO.NS', 'SHIGAN.NS', 'SHILPAMED.NS', 'SHIVALIK.NS', 'SHIVAMAUTO.NS', 'SHIVAMILLS.NS', 'SHIVATEX.NS', 'SHK.NS', 'SHOPERSTOP.NS', 'SHRADHA.NS', 'SHREDIGCEM.NS', 'SHREECEM.NS', 'SHREEPUSHK.NS', 'SHREERAMA.NS', 'SHRENIK.NS', 'SHREYANIND.NS', 'SHREYAS.NS', 'SHRIPISTON.NS', 'SHRIRAMPPS.NS', 'SHUBHLAXMI.NS', 'SHYAMCENT.NS', 'SHYAMMETL.NS', 'SHYAMTEL.NS', 'SIDDHIKA.NS', 'SIEMENS.NS', 'SIGACHI.NS', 'SIGIND.NS', 'SIGMA.NS', 'SIKKO.NS', 'SIL.NS', 'SILGO.NS',
'SILINV.NS', 'SILLYMONKS.NS', 'SILVER.NS', 'SILVERBEES.NS', 'SILVERTUC.NS', 'SIMBHALS.NS', 'SIMPLEXINF.NS', 'SINTERCOM.NS', 'SIRCA.NS', 'SIS.NS', 'SITINET.NS', 'SIYSIL.NS', 'SJS.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SKIPPER.NS', 'SKMEGGPROD.NS', 'SKP.NS', 'SMARTLINK.NS', 'SMCGLOBAL.NS', 'SMLISUZU.NS', 'SMLT.NS', 'SMSLIFE.NS', 'SMSPHARMA.NS', 'SMVD.NS', 'SNOWMAN.NS', 'SOBHA.NS', 'SOFTTECH.NS', 'SOLARA.NS', 'SOLARINDS.NS', 'SOLEX.NS', 'SOMANYCERA.NS', 'SOMATEX.NS', 'SOMICONVEY.NS', 'SONACOMS.NS', 'SONAMCLOCK.NS', 'SONATSOFTW.NS', 'SOTL.NS', 'SOUTHBANK.NS', 'SOUTHWEST.NS', 'SPAL.NS', 'SPANDANA.NS', 'SPARC.NS', 'SPCENET.NS', 'SPECIALITY.NS', 'SPENCERS.NS', 'SPIC.NS',  'SPLIL.NS', 'SPLPETRO.NS', 'SPMLINFRA.NS', 'SPORTKING.NS', 'SPTL.NS', 'SREEL.NS', 'SREIBNPNCD.NS', 'SREINFRA.NS', 'SRF.NS', 'SRHHYPOLTD.NS', 'SRPL.NS',  'SSINFRA.NS', 'SSWL.NS', 'STAR.NS',  'STARHEALTH.NS', 'STARPAPER.NS', 'STARTECK.NS', 'STCINDIA.NS', 'STEELCAS.NS', 'STEELCITY.NS', 'STEELXIND.NS', 'STEL.NS', 'STERTOOLS.NS', 'STLTECH.NS', 'STOVEKRAFT.NS', 'STYLAMIND.NS', 'SUBCAPCITY.NS', 'SUBEXLTD.NS', 'SUBROS.NS', 'SUDARSCHEM.NS', 'SUKHJITS.NS', 'SUMEETINDS.NS', 'SUMICHEM.NS', 'SUMIT.NS', 'SUMMITSEC.NS',  'SUNDARAM.NS', 'SUNDARMFIN.NS', 'SUNDARMHLD.NS', 'SUNDRMBRAK.NS', 'SUNDRMFAST.NS', 'SUNFLAG.NS', 'SUNPHARMA.NS', 'SUNTECK.NS', 'SUNTV.NS', 'SUPERHOUSE.NS', 'SUPERSPIN.NS', 'SUPRAJIT.NS', 'SUPREMEENG.NS', 'SUPREMEIND.NS', 'SUPREMEINF.NS', 'SUPRIYA.NS', 'SURANASOL.NS', 'SURANAT&P.NS', 'SURANI.NS', 'SURYALAXMI.NS', 'SURYAROSNI.NS', 'SURYODAY.NS', 'SUTLEJTEX.NS', 'SUULD.NS', 'SUVEN.NS', 'SUVENPHAR.NS', 'SUVIDHAA.NS', 'SUZLON.NS', 'SUZLONPP.NS', 'SVPGLOB.NS', 'SWANENERGY.NS', 'SWARAJ.NS', 'SWARAJENG.NS', 'SWASTIK.NS', 'SWELECTES.NS', 'SWSOLAR.NS', 'SYMPHONY.NS', 'SYNCOMF.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TAINWALCHM.NS', 'TAJGVK.NS', 'TAKE.NS', 'TALBROAUTO.NS', 'TANLA.NS', 'TANTIACONS.NS', 'TAPIFRUIT.NS', 'TARACHAND.NS', 'TARAPUR.NS', 'TARC.NS', 'TARMAT.NS', 'TARSONS.NS', 'TASTYBITE.NS', 'TATACAPHSG.NS', 'TATACHEM.NS',  'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMETALI.NS', 'TATAMOTORS.NS', 'TATAMTRDVR.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATASTLLP.NS', 'TATVA.NS', 'TBZ.NS', 'TCFSL.NS', 'TCI.NS', 'TCIEXP.NS', 'TCNSBRANDS.NS', 'TCPLPACK.NS', 'TCS.NS', 'TDPOWERSYS.NS', 'TEAMLEASE.NS', 'TECH.NS', 'TECHIN.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEGA.NS', 'TEJASNET.NS', 'TEMBO.NS', 'TERASOFT.NS', 'TEXINFRA.NS', 'TEXMOPIPES.NS', 'TEXRAIL.NS', 'TFCILTD.NS', 'TFL.NS', 'TGBHOTELS.NS', 'THANGAMAYL.NS', 'THEINVEST.NS', 'THEJO.NS', 'THEMISMED.NS', 'THERMAX.NS', 'THOMASCOOK.NS', 'THOMASCOTT.NS', 'THYROCARE.NS', 'TI.NS', 'TIDEWATER.NS', 'TIIL.NS', 'TIINDIA.NS', 'TIJARIA.NS', 'TIL.NS', 'TIMESCAN.NS', 'TIMESGTY.NS', 'TIMETECHNO.NS', 'TIMKEN.NS', 'TINPLATE.NS', 'TIPSFILMS.NS', 'TIPSINDLTD.NS', 'TIRUMALCHM.NS', 'TIRUPATIFL.NS', 'TITAN.NS',
'TMB.NS', 'TNIDETF.NS', 'TNPETRO.NS', 'TNPL.NS', 'TNTELE.NS', 'TOKYOPLAST.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TOTAL.NS', 'TOUCHWOOD.NS', 'TPLPLASTEH.NS', 'TRACXN.NS', 'TREEHOUSE.NS', 'TREJHARA.NS', 'TRENT.NS', 'TRF.NS', 'TRIDENT.NS', 'TRIGYN.NS', 'TRIL.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TRU.NS', 'TTKHLTCARE.NS', 'TTKPRESTIG.NS', 'TTL.NS', 'TTML.NS', 'TV18BRDCST.NS', 'TVSELECT.NS', 'TVSMOTOR.NS', 'TVSSRICHAK.NS', 'TVTODAY.NS', 'TVVISION.NS', 'TWL.NS', 'UBL.NS', 'UCALFUEL.NS', 'UCOBANK.NS', 'UDAICEMENT.NS', 'UFLEX.NS', 'UFO.NS', 'UGARSUGAR.NS', 'UGROCAP.NS', 'UJAAS.NS',  'UJJIVANSFB.NS', 'ULTRACEMCO.NS', 'UMAEXPORTS.NS', 'UMANGDAIRY.NS', 'UMESLTD.NS', 'UNICHEMLAB.NS', 'UNIDT.NS', 'UNIENTER.NS', 'UNIINFO.NS', 'UNIONBANK.NS', 'UNITECH.NS', 'UNITEDPOLY.NS', 'UNITEDTEA.NS', 'UNIVASTU.NS', 'UNIVCABLES.NS', 'UNIVPHOTO.NS', 'UNOMINDA.NS', 'UPL.NS', 'URAVI.NS', 'URJA.NS', 'USASEEDS.NS', 'USHAMART.NS', 'UTIAMC.NS', 'UTIBANKETF.NS', 'UTINEXT50.NS', 'UTINIFTETF.NS', 'UTISENSETF.NS', 'UTISXN50.NS', 'UTTAMSUGAR.NS', 'V2RETAIL.NS', 'VADILALIND.NS', 'VAIBHAVGBL.NS', 'VAISHALI.NS', 'VAKRANGEE.NS', 'VALIANTORG.NS', 'VARDHACRLC.NS', 'VARDMNPOLY.NS', 'VARROC.NS', 'VASA.NS', 'VASCONEQ.NS', 'VASWANI.NS', 'VBL.NS', 'VCL.NS', 'VEDL.NS', 'VEEKAYEM.NS', 'VENKEYS.NS', 'VENUSPIPES.NS', 'VENUSREM.NS', 'VERA.NS', 'VERANDA.NS', 'VERTOZ.NS', 'VESUVIUS.NS', 'VETO.NS', 'VGUARD.NS', 'VHL.NS', 'VIDHIING.NS', 'VIJAYA.NS', 'VIJIFIN.NS', 'VIKASECO.NS', 'VIKASLIFE.NS', 'VIKASPROP.NS', 'VIKASWSP.NS', 'VIMTALABS.NS', 'VINATIORGA.NS', 'VINDHYATEL.NS', 'VINEETLAB.NS', 'VINNY.NS', 'VINYLINDIA.NS', 'VIPCLOTHNG.NS', 'VIPIND.NS', 'VIPULLTD.NS', 'VISAKAIND.NS', 'VISASTEEL.NS', 'VISESHINFO.NS', 'VISHAL.NS', 'VISHNU.NS', 'VISHWARAJ.NS', 'VITAL.NS', 'VIVIANA.NS', 'VIVIDHA.NS', 'VIVIMEDLAB.NS', 'VIVO.NS', 'VLSFINANCE.NS', 'VMARCIND.NS', 'VMART.NS', 'VOLTAMP.NS', 'VOLTAS.NS', 'VRLLOG.NS', 'VSCL.NS', 'VSSL.NS', 'VSTIND.NS', 'VSTTILLERS.NS', 'VTL.NS', 'WABAG.NS', 'WALCHANNAG.NS', 'WANBURY.NS', 'WATERBASE.NS', 'WEALTH.NS', 'WEBELSOLAR.NS', 'WEIZMANIND.NS', 'WELCORP.NS', 'WELENT.NS', 'WELINV.NS',  'WENDT.NS', 'WESTLIFE.NS', 'WEWIN.NS', 'WFL.NS', 'WHEELS.NS', 'WHIRLPOOL.NS', 'WILLAMAGOR.NS', 'WINDLAS.NS', 'WINDMACHIN.NS', 'WINPRO.NS', 'WIPL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'WONDERLA.NS', 'WORTH.NS', 'WSTCSTPAPR.NS', 'XCHANGING.NS', 'XELPMOC.NS', 'XPROINDIA.NS', 'YAARI.NS', 'YESBANK.NS', 'YUKEN.NS', 'ZEEL.NS', 'ZEELEARN.NS', 'ZEEMEDIA.NS', 'ZENITHEXPO.NS', 'ZENITHSTL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZIMLAB.NS', 'ZODIAC.NS', 'ZODIACLOTH.NS', 'ZOMATO.NS', 'ZOTA.NS', 'ZUARI.NS', 'ZUARIIND.NS', 'ZYDUSLIFE.NS', 'ZYDUSWELL.NS']

In [39]:
# Download historical data
def download_data(symbols):
    stock_data = {}
    for symbol in symbols:
        stock_data[symbol] = yf.download(symbol, period="1mo", interval='1d', progress=False)
    return stock_data

# Download data for all stocks
stock_list = scrip_N500
stock_data = download_data(stock_list)

In [40]:
# Define the movement (1 for Up, -1 for Down, 0 for Neutral) based on Open and Close prices
# This code runs for around 18 minutes for N500
def define_movement(df):
    movement = []
    for index, row in df.iterrows():
        if row['Close'] >= row['Open'] * 1.010:
            movement.append(1)
        elif row['Close'] <= row['Open'] * 0.990:
            movement.append(-1)
        else:
            movement.append(0)
    return pd.Series(movement, index=df.index)

# Calculate the correlation between the previous day's movement of stock1 and the next day's movement of stock2
def calculate_correlation(stock_data, stock1, stock2):
    # Define the movements for both stocks
    stock_data[stock1]['Movement'] = define_movement(stock_data[stock1])
    stock_data[stock2]['Movement'] = define_movement(stock_data[stock2])

    # Shift stock2's movement by one day to align with stock1's previous day movement
    stock_data[stock2]['Next_Day_Movement'] = stock_data[stock2]['Movement'].shift(-1)

    # Create a DataFrame to compare movements
    comparison_df = pd.DataFrame({
        f'{stock1}_Movement': stock_data[stock1]['Movement'],
        f'{stock2}_Next_Day_Movement': stock_data[stock2]['Next_Day_Movement']
    }).dropna()

    # Calculate correlation
    correlation = comparison_df.corr().iloc[0, 1]
    return correlation

# Calculate pairwise correlations
correlation_results = []

for stock1, stock2 in permutations(stock_list, 2):
    correlation = calculate_correlation(stock_data, stock1, stock2)
    correlation_results.append((stock1, stock2, correlation))

# Convert to DataFrame for easy sorting and filtering
correlation_df = pd.DataFrame(correlation_results, columns=['Stock1', 'Stock2', 'Correlation'])

In [41]:
# Count the occurrences of each Stock2 with strong correlations
stock2_counts = correlation_df[(correlation_df['Correlation'] >= 0.5) | (correlation_df['Correlation'] <= -0.5)]['Stock2'].value_counts()

# Map these counts back to the original DataFrame
correlation_df['Stock2_Count'] = correlation_df['Stock2'].map(stock2_counts)

In [42]:
# Sort by correlation value in descending order to get the top 5 correlated pairs
top_correlations = correlation_df.sort_values(by='Correlation', ascending=False).head(10)
# Output the top 10 correlated pairs
print("Top 10 highly correlated stock pairs:")
print(top_correlations)

Top 10 highly correlated stock pairs:
               Stock1         Stock2  Correlation  Stock2_Count
199964       TITAN.NS  CARBORUNIV.NS     0.818488           6.0
139398  MOTILALOFS.NS  INDUSINDBK.NS     0.800500          11.0
113146   JKLAKSHMI.NS   EDELWEISS.NS     0.794257          11.0
122645  KAJARIACER.NS        MFSL.NS     0.772141          12.0
217285  ZENSARTECH.NS     EIHOTEL.NS     0.757397          25.0
24728    BALAMINES.NS      VIPIND.NS     0.757397          24.0
118440    JUBLFOOD.NS      MARICO.NS     0.755415          28.0
126219   LAXMIMACH.NS    EIDPARRY.NS     0.750217           5.0
190071  TV18BRDCST.NS  POWERINDIA.NS     0.735867           8.0
72048   GALAXYSURF.NS     EIHOTEL.NS     0.734029          25.0


In [43]:
# Sort by correlation value in descending order to get the top 5 correlated pairs
top_correlations = correlation_df.sort_values(by='Correlation', ascending=True).head(10)
# Output the top 10 correlated pairs
print("Top 10 highly correlated stock pairs:")
print(top_correlations)

Top 10 highly correlated stock pairs:
               Stock1         Stock2  Correlation  Stock2_Count
123263  KARURVYSYA.NS      VGUARD.NS    -0.821995          21.0
58256         DCAL.NS        RVNL.NS    -0.786862          24.0
207292      VGUARD.NS    TATACHEM.NS    -0.777714           2.0
214032     WELCORP.NS         FSL.NS    -0.774294           7.0
177914   SOLARINDS.NS  VINATIORGA.NS    -0.770819           8.0
186708   SUVENPHAR.NS         SCI.NS    -0.767190          32.0
149203  OBEROIRLTY.NS         IGL.NS    -0.764114          28.0
66675   FEDERALBNK.NS     SBICARD.NS    -0.758739          21.0
96676         ISEC.NS  AARTIDRUGS.NS    -0.749043           1.0
186646   SUVENPHAR.NS         NFL.NS    -0.746988          18.0


In [65]:
# Extract each Stock1 that are strongly correlated with Stock2
name_Stock2 = 'CARBORUNIV.NS'   #############------> Only Place to Enter a Name
top_df      = correlation_df[((correlation_df['Correlation'] >= 0.5) | (correlation_df['Correlation'] <= -0.5)) & (correlation_df['Stock2'] == name_Stock2)]
top_df

,Stock1,Stock2,Correlation,Stock2_Count
6625,ATGL.NS,CARBORUNIV.NS,-0.505514,6.0
9427,AKZOINDIA.NS,CARBORUNIV.NS,-0.552122,6.0
29041,BERGEPAINT.NS,CARBORUNIV.NS,0.600920,6.0
78544,GRAPHITE.NS,CARBORUNIV.NS,-0.510853,6.0
198096,RAMCOCEM.NS,CARBORUNIV.NS,0.604940,6.0
199964,TITAN.NS,CARBORUNIV.NS,0.818488,6.0


## ML Model to predict Stock2 Movement

In [62]:
def movement(df):
    conditions = [
        (df['Close'] >= df['Open'] * 1.010),  # Up movement
        (df['Close'] <= df['Open'] * 0.990),  # Down movement
    ]
    choices        = [1, -1]  # 1 for up, -1 for down
    df['Movement'] = np.select(conditions, choices, default=0)  # 0 for neutral
    return df

# List of stock symbols
stock_list    = np.unique(np.concatenate([top_df['Stock2'].unique(), top_df['Stock1'].unique()]))  # List of stocks
stock_symbols = stock_list

# Download the data
stock_data = download_data(stock_symbols)

# Initialize an empty DataFrame to store movements
movement_df = pd.DataFrame()

# Process each stock to calculate movements
for symbol in stock_symbols:
    df                  = stock_data[symbol].copy()  # Extract the individual stock data from the dictionary
    df                  = movement(df)  # Calculate movement
    movement_df[symbol] = df['Movement']  # Store the movement in the new DataFrame

Stock_pred              = name_Stock2
movement_df[Stock_pred] = movement_df[Stock_pred].shift(-1)
movement_df

,AMBER.NS,BASF.NS,CASTROLIND.NS,CRISIL.NS,CROMPTON.NS,DBL.NS,DCAL.NS,GAEL.NS,GICRE.NS,HUDCO.NS,...,LEMONTREE.NS,MMTC.NS,NATCOPHARM.NS,NIACL.NS,PRINCEPIPE.NS,RVNL.NS,SFL.NS,SPARC.NS,SUVENPHAR.NS,SWANENERGY.NS
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-16,-1,0,0,-1,0,0,0,0,1,0,...,-1,-1,1,1,1,-1.0,0,0,1,1
2024-07-18,-1,-1,-1,-1,0,-1,-1,0,-1,-1,...,-1,0,-1,-1,-1,1.0,-1,-1,1,-1
2024-07-19,-1,0,-1,-1,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,1.0,-1,-1,-1,-1
2024-07-22,1,1,1,0,1,1,0,1,1,1,...,1,1,1,1,1,-1.0,0,1,1,0
2024-07-23,-1,0,-1,0,-1,-1,0,0,-1,-1,...,-1,-1,1,-1,0,0.0,1,0,1,-1
2024-07-24,1,1,1,0,1,1,1,1,1,1,...,1,1,1,1,0,-1.0,1,1,1,1
2024-07-25,1,1,1,0,1,0,1,1,-1,1,...,0,1,1,0,0,-1.0,1,0,1,1
2024-07-26,-1,-1,-1,-1,-1,0,0,0,1,-1,...,-1,-1,0,1,0,1.0,-1,-1,0,0
2024-07-29,-1,1,0,1,-1,-1,0,-1,-1,0,...,0,-1,0,-1,0,0.0,0,0,-1,0


In [63]:
X = movement_df.drop(columns=[Stock_pred])
y = movement_df[Stock_pred]

X_train = X.iloc[:-3]
y_train = y.iloc[:-3]

X_val = X.iloc[-3:-1]
y_val = y.iloc[-3:-1]

X_test = X.iloc[-1:]
#y_test = y.iloc[-1:]

In [64]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, Perceptron, PassiveAggressiveClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import statistics
from collections import Counter

# Standardize your features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# List of classifiers to evaluate
classifiers = {
    "Random Forest":           RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting":       GradientBoostingClassifier(random_state=42),
    "AdaBoost":                AdaBoostClassifier(random_state=42),
    "Bagging":                 BaggingClassifier(random_state=42),
    "Extra Trees":             ExtraTreesClassifier(n_estimators=100, random_state=42),
    "Logistic Regression":     LogisticRegression(random_state=42, max_iter=1000),
    "Ridge Classifier":        RidgeClassifier(random_state=42),
    "SGD Classifier":          SGDClassifier(random_state=42),
    "Perceptron":              Perceptron(random_state=42),
    "Passive-Aggressive":      PassiveAggressiveClassifier(random_state=42),
    "SVM":                     SVC(random_state=42),
    "Linear SVM":              LinearSVC(random_state=42, max_iter=10000, dual=False),
    "NuSVC":                   NuSVC(random_state=42),
    "K-Neighbors":             KNeighborsClassifier(),
    "Decision Tree":           DecisionTreeClassifier(random_state=42),
    "Naive Bayes (Gaussian)":  GaussianNB(),
    "Naive Bayes (Bernoulli)": BernoulliNB(),
    "Neural Network (MLP)":    MLPClassifier(random_state=42, max_iter=2000, alpha=0.1)
}

ML_result = []

# Define the width for alignment
width = max(len(name) for name in classifiers.keys()) + 1

# Loop through classifiers, train, predict, and print accuracy
for name, clf in classifiers.items():
    try:
        # Train the model on scaled data
        clf.fit(X_train_scaled, y_train)

        # Predict on the validation and test data
        y_pred = clf.predict(X_val_scaled)
        y_test = clf.predict(X_test_scaled)

        ML_result.append(y_test)

        # Evaluate the model
        accuracy = accuracy_score(y_val, y_pred)
        y_val_str = ', '.join(map(str, y_pred))
        y_test_str = ', '.join(map(str, y_test))

        # Print the accuracy with aligned colons
        print(f'{name:<{width}} Accuracy: {accuracy:.2f}  Validation: {y_val_str}   Prediction: {y_test_str}')

    except Exception as e:
        pass

# Flatten the list of arrays and convert to a list of values
values = [arr[0] for arr in ML_result if len(arr) > 0]

# Count the frequency of each value
frequency_count = Counter(values)

# Print the frequencies
print("\nFrequencies of predictions:")
print(f" -1: {frequency_count[-1]}")
print(f"  0: {frequency_count[0]}")
print(f"  1: {frequency_count[1]}")

# Print the mode result
print("\nML Voting Result for", Stock_pred, "for", str(X_test.index[0].date() + timedelta(days=1)), "is:", statistics.mode(values))

Random Forest            Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
Gradient Boosting        Accuracy: 0.50  Validation: -1.0, -1.0   Prediction: -1.0
AdaBoost                 Accuracy: 0.00  Validation: 0.0, -1.0   Prediction: -1.0
Bagging                  Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
Extra Trees              Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
Logistic Regression      Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
Ridge Classifier         Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
SGD Classifier           Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
Perceptron               Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
Passive-Aggressive       Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
SVM                      Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
Linear SVM               Accuracy: 0.50  Validation: 1.0, 1.0   Prediction: -1.0
K-Neighbors              